In [4]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is inc

In [14]:
!pip install pypdf

In [21]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 20.6 MB/s eta 0:00:00


In [24]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

docs = []
docs += PyPDFLoader("/content/Workshop 5 FAQ_V1.pdf").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

db = FAISS.from_documents(chunks, embeddings)
db.save_local("vectorstore")

print("✅ Knowledge base indexed")


✅ Knowledge base indexed


In [25]:
%%writefile rag.py
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

VECTOR_DIR = "/content/vectorstore"

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

db = FAISS.load_local(
    VECTOR_DIR,
    embeddings,
    allow_dangerous_deserialization=True
)

def retrieve(query: str, k: int = 3):
    docs = db.similarity_search(query, k=k)
    return [
        {
            "content": d.page_content,
            "source": d.metadata.get("source", "unknown")
        }
        for d in docs
    ]


Writing rag.py


In [26]:
%%writefile tools.py
from rag import retrieve

# MCP-style tool (tool + contract)
def search_kb(query: str):
    return retrieve(query)


Writing tools.py


In [33]:
%%writefile graph.py
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from tools import search_kb

# --------------------
# STATE SCHEMA
# --------------------
class AgentState(TypedDict):
    question: str
    docs: List[dict]
    draft: str
    final: str

# --------------------
# LLM
# --------------------
llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

# --------------------
# GRAPH NODES
# --------------------
def retrieve_node(state: AgentState):
    return {
        "docs": search_kb(state["question"])
    }

def draft_node(state: AgentState):
    context = "\n".join(d["content"] for d in state["docs"])
    prompt = f"""
Answer using ONLY the context below.

Context:
{context}

Question:
{state["question"]}
"""
    return {"draft": llm.invoke(prompt)}

def cite_node(state: AgentState):
    sources = {d["source"] for d in state["docs"]}
    final = state["draft"] + "\n\nSources:\n" + "\n".join(sources)
    return {"final": final}

# --------------------
# BUILD GRAPH
# --------------------
graph = StateGraph(AgentState)

graph.add_node("retrieve", retrieve_node)
graph.add_node("draft", draft_node)
graph.add_node("cite", cite_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "draft")
graph.add_edge("draft", "cite")
graph.add_edge("cite", END)

app = graph.compile()


Overwriting graph.py


In [44]:
from graph import app

result = app.invoke({
    "question": "How to access the workshop"
})

print("\n Final Answer:\n")
print(result["final"])



 Final Answer:

Register for the workshop using the "Join event" button.

Sources:
/content/Workshop 5 FAQ_V1.pdf
